In [2]:
import pandas as pd 
data=pd.read_csv('bay_area_rentals_sample.csv')
data.head()

,Property_ID,County,Address,Zip_Code,Crime_Rate,Bedrooms,Bathrooms,Walkability_Score,Floor,New_Construction,Renovated,Washer_Dryer,Trash_Service,Average_Rent
0,5f94c7e7-887b-459c-a918-6324df20437d,Solano,"068 Nielsen Rapids, Lake Ashley, MT 52153",99651,3.5,4,3,75,4,False,False,False,NOT_INCLUDED,$2843
1,3c3363c8-633b-4367-bcb0-31588957cd68,San Mateo,"31861 Ann Isle Suite 931, Andreaton, PW 04917",50927,4.5,4,3,77,4,False,False,True,INCLUDED,$3672
2,1f79d1c2-eed6-43e3-8097-3f11b2671dfa,San Francisco,"48666 Freeman Oval, Port John, WY 62849",55080,4.6,2,2,90,3,True,False,True,INCLUDED,$2749
3,b004b54e-3a86-4718-89d8-928b821ade03,Napa,"USNV Garcia, FPO AE 14574",31560,3.8,2,2,74,1,True,False,True,INCLUDED,$3738
4,a0997700-6d3e-47e3-b744-b37dec531eca,San Mateo,"75822 Hernandez Mews, North Brendabury, NE 76892",4718,2.8,2,3,85,2,False,True,True,INCLUDED,$3328


In [31]:
all_features=data.columns.to_list()
all_features

['Property_ID',
 'County',
 'Address',
 'Zip_Code',
 'Crime_Rate',
 'Bedrooms',
 'Bathrooms',
 'Walkability_Score',
 'Floor',
 'New_Construction',
 'Renovated',
 'Washer_Dryer',
 'Trash_Service',
 'Average_Rent']

In [32]:
rel_features=['County',
 'Bedrooms',
 'Bathrooms',
 'Walkability_Score',
 'Renovated',
 'Washer_Dryer',
 'Average_Rent']
fin_df=data[rel_features]
fin_df['Renovated']=fin_df['Renovated'].apply(lambda x: 1 if True else 0)
fin_df['Washer_Dryer']=fin_df['Washer_Dryer'].apply(lambda x: 1 if True else 0)
fin_df['Average_Rent']=fin_df['Average_Rent'].apply(lambda x: int(x.replace('$','')))
fin_df.head()

/var/folders/_1/4qth7z751wq149nn6dw5d2r40000gn/T/ipykernel_35175/1585065529.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin_df['Renovated']=fin_df['Renovated'].apply(lambda x: 1 if True else 0)
/var/folders/_1/4qth7z751wq149nn6dw5d2r40000gn/T/ipykernel_35175/1585065529.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin_df['Washer_Dryer']=fin_df['Washer_Dryer'].apply(lambda x: 1 if True else 0)
/var/folders/_1/4qth7z751wq149nn6dw5d2r40000gn/T/ipykernel_35175/1585065529.py:11: SettingWithCopyWar

,County,Bedrooms,Bathrooms,Walkability_Score,Renovated,Washer_Dryer,Average_Rent
0,Solano,4,3,75,1,1,2843
1,San Mateo,4,3,77,1,1,3672
2,San Francisco,2,2,90,1,1,2749
3,Napa,2,2,74,1,1,3738
4,San Mateo,2,3,85,1,1,3328


In [33]:
from sklearn.preprocessing import OneHotEncoder
categorical_columns = fin_df.select_dtypes(include=['object']).columns.tolist()
print(categorical_columns)
encoder = OneHotEncoder(sparse_output=False)
one_hot_encoded = encoder.fit_transform(fin_df[categorical_columns])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))
one_hot_df=one_hot_df.astype('int')
df_encoded = pd.concat([fin_df, one_hot_df], axis=1)
df_encoded = df_encoded.drop(categorical_columns, axis=1)
df_encoded.head()

['County']


,Bedrooms,Bathrooms,Walkability_Score,Renovated,Washer_Dryer,Average_Rent,County_Alameda,County_Contra Costa,County_Marin,County_Napa,County_San Francisco,County_San Mateo,County_Santa Clara,County_Solano,County_Sonoma
0,4,3,75,1,1,2843,0,0,0,0,0,0,0,1,0
1,4,3,77,1,1,3672,0,0,0,0,0,1,0,0,0
2,2,2,90,1,1,2749,0,0,0,0,1,0,0,0,0
3,2,2,74,1,1,3738,0,0,0,1,0,0,0,0,0
4,2,3,85,1,1,3328,0,0,0,0,0,1,0,0,0


In [46]:
X=df_encoded.drop('Average_Rent', axis=1)
y=df_encoded['Average_Rent']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2)

In [48]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
model=LinearRegression()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
print(mean_squared_error(y_test,y_pred))

335368.99588245084


In [49]:
import pickle
pickle.dump(model, open('rent_prediction_bay_area', 'wb'))

In [51]:
#Model Optimization
from sklearn.preprocessing import Normalizer
X_transform=Normalizer().fit_transform(X)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X_transform,y,test_size=0.2)
model=LinearRegression()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
print(mean_squared_error(y_test,y_pred))

334665.460953578


In [52]:
y_test

5306     3605
6845     3183
4959     3045
18360    3875
12863    2582
         ... 
11853    3250
18968    2050
8005     3849
8287     3365
15672    3562
Name: Average_Rent, Length: 4000, dtype: int64

In [53]:
#normality is important in LSE 


array([3003.662625, 2991.662625, 2991.662625, ..., 3013.662625,
       3011.662625, 2993.662625])